In [3]:
from bokeh.io import push_notebook, show, output_notebook, curdoc
from bokeh.palettes import brewer , inferno, viridis, cividis , plasma
from bokeh.models import ColumnDataSource, Select, ColorBar, BasicTicker, LinearColorMapper, Div, Slider, Spinner, Dropdown
from bokeh.layouts import row, gridplot, column, layout
from bokeh.plotting import figure, show, output_file, output_notebook
from random import random
from time import time
import numpy as np
from bokeh.core.validation import silence
from bokeh.core.validation.warnings import EMPTY_LAYOUT, MISSING_RENDERERS


from demo import evaluate_polynomial, run_demo

output_notebook()

#run_demo()

Loading BokehJS ...

In [4]:
plot_title = "Demo plot"
CALLBACK_TIME_MS = 500
plot_x=-1  # plot axis parameters
plot_y=-1  # plot axis parameters
plot_dw=2 # plot axis parameters
plot_dh=2 # plot axis parameters

In [5]:
def bkapp(doc):
    "define app"
    #TOOLS define the default Bokeh graph tools 
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"
    
    plot = figure(title= plot_title, tools=TOOLS,  plot_width=400, plot_height=400)  
    
    #plot.sizing_mode = 'scale_width'
    plot.xaxis.axis_label = 'Real axis'
    plot.yaxis.axis_label = 'Imaginary axis'
    
    
    # import variables from variables file
    from demo_variables import generate_variables
    measurement_variables = generate_variables()



    x , y  = np.linspace(plot_x, plot_x + plot_dw, 100) , np.linspace(plot_y, plot_y + plot_dh, 100)
    xv, yv = np.meshgrid(x, y, sparse=True)
    initial_image = evaluate_polynomial(xv, yv, measurement_variables)
    
    # record time to increase noise overtime
    start_time = time()


    source = ColumnDataSource(dict(Hs=[]))
     
    color_palette = inferno(80) # max : 256
 
    color_mapper = LinearColorMapper(palette=color_palette, 
                                    low=np.min(np.abs(initial_image)), 
                                    high=np.max(np.abs(initial_image)))
    

    img = plot.image(image='Hs', 
                 x=plot_x, 
                 y=plot_y, 
                 dw=plot_dw, 
                 dh=plot_dh, 
                 color_mapper=color_mapper,  
                 source = source)
    
    color_bar = ColorBar(color_mapper=color_mapper,  ticker=BasicTicker(desired_num_ticks=len(color_palette)+1),)
    
    dummy = figure(height=300, width=100, toolbar_location=None, min_border=0, outline_line_color=None)
    dummy.add_layout(color_bar, 'right')
    
    def create_value():
        P = evaluate_polynomial(xv, yv, measurement_variables)
        return np.abs(P + P * 0.03 * np.sqrt(time() - start_time)* np.random.rand(* P.shape) ) 
        

    def update(new_color_palette = None):
        
        newimage= create_value()
        new_data = dict(Hs=[newimage])
        source.stream(new_data , rollover=1)
        
        if new_color_palette:
            
            img.glyph.color_mapper = LinearColorMapper(palette=new_color_palette, 
                                    low=np.min(newimage), 
                                    high=np.max(newimage))
            
            color_bar = ColorBar(color_mapper=img.glyph.color_mapper,  ticker=BasicTicker(desired_num_ticks=len(color_palette)+1),)
    
            lay_out.children.pop()
            dummy = figure(height=300, width=100, toolbar_location=None, min_border=0, outline_line_color=None)
            dummy.add_layout(color_bar, 'right')
            lay_out.children.append(dummy)
         
    
    color_dropdown = Dropdown(label='Select plot color', menu=['brewer (red/blue)', 'brewer (Purple/Orange)', 'plasma' , 'inferno', 'viridis', 'cividis' ])


    def dropdown_handler(event):
        #handle dropdown input
        color_palette = None
        if event.item == 'brewer (red/blue)':
            color_palette = brewer['RdYlBu'][10]
        elif event.item == 'brewer (Purple/Orange)':
            color_palette = brewer['PuOr'][10]
        elif event.item == "inferno":
            color_palette = inferno(80)
        elif event.item == "viridis":
            color_palette = viridis(80)
        elif event.item == 'plasma':
            color_palette = plasma(80)
        else:
            color_palette = cividis(80)
        
        update(new_color_palette=color_palette)
        
    color_dropdown.on_click(dropdown_handler)
    
    lay_out = layout( 
        row(column(*[v.ui_row for v in measurement_variables]), column(color_dropdown,plot,) ,), dummy,
    )
    
    silence(EMPTY_LAYOUT, True)
    silence(MISSING_RENDERERS, True)
    
    doc.theme = "dark_minimal"
    doc.title = "Instrument UI"
    doc.add_root(lay_out)
    doc.add_periodic_callback(update, CALLBACK_TIME_MS)
  

In [6]:
show(bkapp)

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=1073&bokeh-absolute-url=http://localhost:56668&resources=none (::1)
HTTPServerRequest(protocol='http', host='localhost:56668', method='GET', uri='/autoload.js?bokeh-autoload-element=1073&bokeh-absolute-url=http://localhost:56668&resources=none', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\web.py", line 1704, in _execute
    result = await result
  File "C:\ProgramData\Anaconda3\lib\site-packages\bokeh\server\views\autoload_js_handler.py", line 60, in get
    session = await self.get_session()
  File "C:\ProgramData\Anaconda3\lib\site-packages\bokeh\server\views\session_handler.py", line 120, in get_session
    session = await self.application_context.create_session_if_needed(session_id, self.request, token)
  File "C:\ProgramData\Anaconda3\lib\site-packages\bokeh\server\contexts.py", line 218, in create_session_if_ne